<a href="https://colab.research.google.com/github/ijusplab/implantacao-automatica/blob/master/Implantacao_LOAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de Planilha de Execução Semi-Automática para B-87 e B-88

**ENTRADA DE DADOS**

1) Primeiro entre com o número do lote e pressione `CTRL-ENTER`.

2) Se o o botão de `upload` não aparecer em poucos segundos, tente executar a célula clicando no símbolo ▶️.

3) Carregue os arquivos `pdf` das sentenças proferidas em relação ao lote.

In [0]:
LOTE = "" #@param {type:"string"}

!pip install PyPDF2

from google.colab import files
import pandas as pd
import re, os, PyPDF2
from IPython.display import display, HTML

def get_text_from_pdf(file_name):
  pdfFileObj = open(file_name, 'rb')
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  content = []
  for num in range(pdfReader.numPages):
    page = pdfReader.getPage(num)
    try:
      text = page.extractText()
    except:
      text = ''
    content.append(text)
  return ''.join(content)

def obterEspecie(val):
  if re.compile('IDOS[AO]|88').search(val.upper()):
    return 88
  else:
    return 87

class PDFReader:
  def __init__(self):
    _files = files.upload()
    self.sources = []
    if len(_files) > 0:
      for name, content in _files.items():
        if re.match('^.+\.(pdf)$', name):
          self.sources.append({
              'name': name,
              'content': get_text_from_pdf(name)
          })
        else:
          print('>>>> ATENÇÃO')
          print('>>>> O arquivo ' + name + ' não é um pdf')
  
  def isEmpty(self):
    return len(self.sources) == 0

  def read(self):
    patterns = {
        'processo': re.compile('PROCESSO:\s*?(\d{7}\-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4})', re.DOTALL),
        'especie': re.compile('ESP[ÉE]CIE DO NB:\s([À-ü]?.*)RMI', re.DOTALL),
        'DIB': re.compile('DIB:\s*?(\d{1,2}[/\-.]\d{1,2}[/\-.]\d{2,4})', re.DOTALL),
        'DIP': re.compile('DIP:\s*?(\d{1,2}[/\-.]\d{1,2}[/\-.]\d{2,4})', re.DOTALL)
    }
    if self.isEmpty():
      print('>>>> Nenhum dado encontrado')
      return
    columns = patterns.keys()
    data = {}
    for label, pattern in patterns.items():
      column = []
      for source in self.sources:
        match = pattern.search(source['content'])
        if match:
          column.append(match.group(1))
        else:
          column.append('')
          print('>>>> Atributo "' + label + '" não encontrado em ' + source['name'])
      data[label] = column
    
    self.df = pd.DataFrame(data, columns=columns)
    self.df = self.df.fillna('')
    self.df['especie'] = self.df['especie'].apply(obterEspecie)
    self.csv = self.df.to_csv()

  def show(self):
    display(HTML(self.df.to_html()))

  def download(self):
    with open(LOTE + '.csv', 'w') as file:
      file.write(self.df.to_csv())
    files.download(LOTE + '.csv')

reader = PDFReader()
reader.read()
reader.show()








**BAIXAR CSV**

4) Agora basta executar a célula abaixo para baixar o arquivo `csv`.

In [0]:
reader.download()